# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os

from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# constants

MODEL_GPT = "gpt-4o-mini"
MODEL_LLAMA = "llama3.2"

In [ ]:
# set up environment
# api key has to be in the .env file in the project root directory
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
assert openai_api_key and openai_api_key.startswith("sk-proj-") and len(openai_api_key) > 10, \
       "something is wrong with your api key"

In [ ]:
# user example question and system prompt

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

system_prompt = """You are a helpful learning assistant.
You help students to understand their subject matter.
You give easy-to-understand and well-structured answers, favouring short sentences and bullet points over long, rambling explanations.
You like to explain using examples, which are always limited to the essentials.
You don't waste time with polite phrases.
Answer in markdown."""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]

In [ ]:
# Get model to answer, with streaming
def get_display_answer(api: OpenAI, model: str, messages: list[dict]):
    stream = api.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        # response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
# Get gpt-4o-mini to answer
openai_api = OpenAI()

get_display_answer(openai_api, MODEL_GPT, messages)

In [ ]:
# Get Llama 3.2 to answer
# ollama has to be installed (https://ollama.com) and the model has to be downloaded with
# ollama pull llama3.2 (2GB)
ollama_api = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

get_display_answer(ollama_api, model=MODEL_LLAMA, messages=messages)